## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Step 1: Compute the camera calibration using chessboard images

In [38]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space (real undistorted chessboard corners)
               # These are the known object coordinates of the chessboard corners
               # x, y, and z from the top left corner (0,0,0) to (x, y, 0) --> flat 
imgpoints = [] # 2d points in image plane. (coordinates of the corners)

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Array to store valid_imgages
valid_imgs = []

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        valid_imgs.append(fname)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
cv2.destroyAllWindows()

## Step 2: Correct for Distortion (and save undistorted images)

In [85]:
import re
undist_imgs = []
for i in range(0, len(valid_imgs)):
    img = cv2.imread(valid_imgs[i])
    img_num = ''.join(re.findall("\d",(valid_imgs[i])))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imshow('img',undist)
    cv2.waitKey(500)
    undist_path = str('camera_undist'+'/undistort'+img_num+'.jpg')
    undist_imgs.append(undist_path)
    cv2.imwrite(undist_path, undist)

## Step 3: Perspective Transform (and save transformed images)

In [88]:
nx = 9
ny = 6
trans_paths = []
for i in range(0, len(undist_imgs)):
    img = cv2.imread(undist_imgs[i])
    img_num = ''.join(re.findall("\d",(undist_imgs[i]))) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    if ret == True:
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        offset = 100
        img_size = (gray.shape[1], gray.shape[0])
        src = np.float32([corners[0], corners[nx-1],corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset],
                          [img_size[0]-offset, img_size[1]-offset],
                          [offset, img_size[1]-offset]])
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(img, M, img_size)
        cv2.imshow('img',warped)
        cv2.waitKey(500)
        trans_path = str('camera_transform'+'/transform'+img_num+'.jpg')
        trans_paths.append(trans_path)
        cv2.imwrite(trans_path, warped)

# And so on and so forth...

cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None) # returns ret, 
#mtx (camera matrix we need to transform 3D obj points to 2D image points)
#dist (distortion coefficients)
#rvecs (rotation vectors)
#tvecs (translation vectors)

dst = cv2.undistort(img, mtx, dist, None, mtx)